### Surface

In [19]:
import numpy as np

# Parameters

Lu0 = 1
Lv0 = 1

Nu = 20
Nv = 20

N_rand_V_u = 5
N_rand_V_v = 5

N_rand_Omg_u = 5
N_rand_Omg_v = 5

rand_V_var = 1e0
rand_Omg_var = 1e0

# Initial conditions

thu0 = np.zeros((3, Nu, Nv))
thv0 = np.zeros((3, Nu, Nv))
piu0 = np.zeros((3, Nu, Nv))
piv0 = np.zeros((3, Nu, Nv))

thu0[1] = 1
thv0[2] = 1

piu0[1] = 0
piv0[0] = 0

# Construct material mesh grid

def get_grid_1D(Mm, L):
    N = Mm - 1
    x = np.cos(np.pi*np.arange(0,N+1)/N)
    x = L*(1-x)/2
    return x

us = get_grid_1D(Nu, Lu0)
vs = get_grid_1D(Nv, Lv0)
grid_U, grid_V = np.meshgrid(us, vs, indexing='ij')

# Define random velocity and and angular velocity

rand_V_coeffs = np.zeros((3, N_rand_V_u, N_rand_V_v))
rand_Omg_coeffs = np.zeros((3, N_rand_Omg_u, N_rand_Omg_v))

for di in range(dim):
    for i in range(N_rand_V_u):
        for j in range(N_rand_V_v):
            rand_V_coeffs[di,i,j] = rand_V_var * np.random.normal()
            rand_Omg_coeffs[di,i,j] = rand_Omg_var * np.random.normal()
            
# Compute velocity field

V_field = np.zeros((3, Nu, Nv))
for di in range(dim):
    for i in range(N_rand_V_u):
        for j in range(N_rand_V_v):
            V_field[di] += rand_V_coeffs[di,i,j] * np.sin((i - 0.5)*np.pi*grid_U/Lu0) * np.sin((j - 0.5)*np.pi*grid_V/Lv0) / ( (i-0.5) * (j - 0.5) * np.pi**2 )

# Compute angular velocity field

Omg_field = np.zeros((3, Nu, Nv))
for di in range(dim):
    for i in range(N_rand_Omg_u):
        for j in range(N_rand_Omg_v):
            Omg_field[di] += rand_Omg_coeffs[di,i,j] * np.sin((i - 0.5)*np.pi*grid_U/Lu0) * np.sin((j - 0.5)*np.pi*grid_V/Lv0) / ( (i-0.5) * (j - 0.5) * np.pi**2 )


In [20]:
lmbd = 1

T = 0.1

cn_K = np.random.random((3,3))

cn_K = cn_K + cn_K.T
cn_K += np.eye(3) * 3
cn_K /= np.max(np.linalg.eigvals(cn_K))

cn_K *= 1e-2


cn_Eps = np.random.random((3,3))

cn_Eps = cn_Eps + cn_Eps.T
cn_Eps += np.eye(3) * 3
cn_Eps /= np.max(np.linalg.eigvals(cn_Eps))

cn_Eps *= 1e-3

In [21]:
def random_pos_def_matrix():
    mat = np.random.random((3,3))
    mat = mat + mat.T
    mat += np.eye(3) * 3
    mat /= np.max(np.linalg.eigvals(mat))
    mat *= 1e-2
    return mat

In [37]:
# Parameters
lmbd = 1
alpha = 1
T = 0.1
Nu = 200

def random_pos_def_matrix():
    mat = np.random.random((3,3))
    mat = mat + mat.T
    mat += np.eye(3) * 3
    mat /= np.max(np.linalg.eigvals(mat))
    mat *= 1e-2
    return mat

# Generate random K1 matrix
K1 = random_pos_def_matrix()

# Generate random K2 matrix
K2 = random_pos_def_matrix()

# Generate random moment of inertia
mom_I = random_pos_def_matrix()

# Generate random center-line

L0 = 1
N_random_curve_modes = 3
mu_random_curve = 0
sigma_random_curve = 0.3

def get_grid_1D(Mm, L):
    N = Mm - 1
    x = np.cos(np.pi*np.arange(0,N+1)/N)
    x = L*(1-x)/2
    return x

def generate_random_periodic_function(dim, us, L, N, mu, sigma):
    fs = np.zeros((dim, len(us)))
    dfs = np.zeros((dim, len(us)))
    
    for i in range(dim):
        fs[i] += np.random.normal(mu, sigma)/2
        
        for j in range(1, N):
            a, b= np.random.normal(mu, sigma), np.random.normal(mu, sigma)
            
            fs[i] += a * np.cos((2*np.pi/L)*j*us) / (j*np.pi)
            fs[i] += b * np.sin((2*np.pi/L)*j*us) / (j*np.pi)
            
            dfs[i] += -(a * (2*np.pi/L)*j / (j*np.pi)) * np.sin((2*np.pi/L)*j*us)
            dfs[i] += (b * (2*np.pi/L)*j / (j*np.pi)) * np.cos((2*np.pi/L)*j*us)
            
    return fs, dfs

us_grid = get_grid_1D(Nu, L0)

R0_raw, dR0_raw = generate_random_periodic_function(3, us_grid, L0, N_random_curve_modes, mu_random_curve, sigma_random_curve)


In [49]:
N_R_modes = 3
mu_R = 3
sigma_R = 0.5

def eul2rot(rho, phi, psi) :
    R = np.zeros((3, 3, rho.shape[-1]))
    R[0,0] = np.cos(phi)*np.cos(psi)
    R[0,1] = np.sin(rho)*np.sin(phi)*np.cos(psi) - np.sin(psi)*np.cos(rho)
    R[0,2] = np.sin(phi)*np.cos(rho)*np.cos(psi) + np.sin(rho)*np.sin(psi)
    
    R[1,0] = np.sin(psi)*np.cos(phi)
    R[1,1] = np.sin(rho)*np.sin(phi)*np.sin(psi) + np.cos(rho)*np.cos(psi)
    R[1,2] = np.sin(phi)*np.sin(psi)*np.cos(rho) - np.sin(alprhoha)*np.cos(psi)
    
    R[2,0] = -np.sin(phi)
    R[2,1] = np.sin(rho)*np.cos(phi)
    R[2,2] = np.cos(rho)*np.cos(phi)

    return R

us_grid = get_grid_1D(Nu, L0)
R_rho = generate_random_periodic_function(1, us, L0, N_rot_modes, mu_rot, sigma_rot)[0]
R_phi = generate_random_periodic_function(1, us, L0, N_rot_modes, mu_rot, sigma_rot)[0]
R_psi = generate_random_periodic_function(1, us, L0, N_rot_modes, mu_rot, sigma_rot)[0]

R = eul2rot(R_rho, R_phi, R_psi)

array([[[ 0.81294947,  0.87125653,  0.96598733,  0.97043181,
          0.97317647,  0.93722893,  0.22642584,  0.13013238,
          0.53118082,  0.46762721,  0.38672944,  0.67477254,
          0.93545804,  0.93409861,  0.83913343,  0.16987529,
          0.00633052,  0.45585757,  0.74038039,  0.81294947],
        [ 0.56465237,  0.48886762,  0.21161511, -0.14955307,
         -0.18450401,  0.29673977,  0.21870105,  0.08359697,
          0.07081914,  0.17981935,  0.14164167, -0.06312097,
          0.08185362,  0.23348808,  0.03056501,  0.32577259,
          0.50327985,  0.66871387,  0.62285138,  0.56465237],
        [ 0.14241085,  0.04382354, -0.1486187 , -0.18946234,
         -0.13742572,  0.18315963,  0.94915815,  0.98796615,
          0.84429354,  0.86544196,  0.91125077,  0.73532154,
          0.3438288 ,  0.27007979,  0.54306617,  0.93006161,
          0.86410029,  0.58738032,  0.25277074,  0.14241085]],

       [[-0.54040957, -0.47029735, -0.2527916 ,  0.00712407,
          0.0728919

In [45]:
R_psi.shape

(1, 20)

In [ ]:
frame_rot_th = np.sin(2*np.pi*path_handler_render.grid/L0) * frame_rot_ampl
frame_rot_phi = np.cos(2*np.pi*path_handler_render.grid/L0) * frame_rot_ampl
frame_rot_psi = np.sin(2*np.pi*path_handler_render.grid/L0) * frame_rot_ampl

In [ ]:
frame_rotation = eul2rot([frame_rot_th, frame_rot_phi, frame_rot_psi])
du_frame_rotation = np.zeros(frame_rotation.shape)
for i in range(3):
    du_frame_rotation[i,:] = path_handler_render.diff_f( frame_rotation[i,:] )

transformed_pi0_hat_render = np.einsum('iju,kju->iku', du_frame_rotation, frame_rotation)
transformed_pi0_hat_render += np.einsum('iju,jku,lku->ilu', frame_rotation, pi0_hat_render, frame_rotation)
transformed_pi0_render = hat_mat_to_vec(transformed_pi0_hat_render)

transformed_th0_render = np.einsum('iju,ju->iu', frame_rotation, th0_render)